In [2]:
import pandas as pd
from collections import defaultdict
import warnings
import visualize
import neat
import multiprocessing
import os
import matplotlib.pyplot as plt

import numpy as np

In [15]:
df = pd.read_csv('shuttle.trn',header=None,sep = '\s')

In [16]:
X = df.iloc[:,:-1].to_numpy()
y = df.iloc[:,-1].to_numpy()

for i in range(7):
    y[y == (i + 1)] = i

In [17]:
t = 3000
y_new = y[t:4350+t]
list_y = y_new.tolist()

dict = {}
for key in list_y:
    dict[key] = dict.get(key, 0) + 1
print(dict)

{0: 3411, 3: 670, 4: 249, 2: 15, 1: 2, 6: 2, 5: 1}


In [8]:
def softmax(x, axis=1):
    # 计算每行的最大值
    row_max = x.max(axis=axis)
 
    # 每行元素都需要减去对应的最大值，否则求exp(x)会溢出，导致inf情况
    row_max=row_max.reshape(-1, 1)
    x = x - row_max
 
    # 计算e的指数次幂
    x_exp = np.exp(x)
    x_sum = np.sum(x_exp, axis=axis, keepdims=True)
    s = x_exp / x_sum
    return s

In [18]:
def eval_genomes(genomes, config):
    for genome_id, genome in genomes:
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        
        outputs = []
        for xi in x_inputs:
            output = net.activate(xi)
            outputs.append(output)
        
        
        px_outputs = softmax(np.array(outputs).reshape(samplesize, number_of_classification), axis=1)
        # the index of maximum in each line
        pred_outputs = np.argmax(px_outputs, axis = 1)
        # replace index to real number, from higher number to lower number!!
        for i in range(number_of_classification):
            index = number_of_classification - i - 1
            pred_outputs[pred_outputs==index] = numbers[index]
        
        real_outputs = np.array(x_outputs).reshape(samplesize,)
        
        acc = np.sum(pred_outputs == real_outputs)/samplesize

        genome.fitness = acc

In [19]:
X = X[t:4350+t]
y = y[t:4350+t]

In [20]:
numbers = [*range(7)] # number 2 > number 1
number_of_classification = 7


list_y = y.tolist()
digits_indexes = []
for digit in numbers:
    li = [i for i in range(len(list_y)) if list_y[i] == digit]
    digits_indexes.extend(li)

samplesize = X[digits_indexes].shape[0]
x_inputs =[tuple(c) for c in X[digits_indexes].tolist()]
x_outputs = [tuple(c) for c in y[digits_indexes].reshape(samplesize,1).tolist()]

In [ ]:
def run(config_file):
    # Load configuration.
    config = neat.Config(
        neat.DefaultGenome,
        neat.DefaultReproduction,
        neat.DefaultSpeciesSet,
        neat.DefaultStagnation,
        config_file,
    )

    # Create the population, which is the top-level object for a NEAT run.
    p = neat.Population(config)

    # add a stdout reporter to show progress in the terminal
    reporter = neat.StdOutReporter(False)
    p.add_reporter(reporter)
    stats = neat.StatisticsReporter()
    p.add_reporter(stats)
    #checkpointer = neat.Checkpointer(100)
    #p.add_reporter(checkpointer)
    # Run for up to 300 generations.
    winner = p.run(eval_genomes, 3000)

    return [stats, winner]


samplesize = len(x_outputs)

local_dir = os.getcwd()
config_path = os.path.join(local_dir, "config-feedforward-shuttle7")
[stats, winner] = run(config_path)

config = neat.Config(
    neat.DefaultGenome,
    neat.DefaultReproduction,
    neat.DefaultSpeciesSet,
    neat.DefaultStagnation,
    config_path,
)
winner_net = neat.nn.FeedForwardNetwork.create(winner, config)


 ****** Running generation 0 ****** 

Population's average fitness: 0.19833 stdev: 0.26946
Best fitness: 0.86046 - size: (7, 6) - species 2 - id 32
Average adjusted fitness: 0.200
Mean genetic distance 2.641, standard deviation 0.317
Population of 200 members in 22 species
Total extinctions: 0
Generation time: 30.686 sec

 ****** Running generation 1 ****** 

Population's average fitness: 0.39432 stdev: 0.33872
Best fitness: 0.86046 - size: (7, 6) - species 11 - id 32
Average adjusted fitness: 0.384
Mean genetic distance 2.754, standard deviation 0.368
Population of 200 members in 26 species
Total extinctions: 0
Generation time: 34.517 sec (32.602 average)

 ****** Running generation 2 ****** 

Population's average fitness: 0.48408 stdev: 0.34253
Best fitness: 0.86046 - size: (7, 6) - species 22 - id 32
Average adjusted fitness: 0.452
Mean genetic distance 2.757, standard deviation 0.405
Population of 200 members in 27 species
Total extinctions: 0
Generation time: 35.026 sec (33.410 a

Population's average fitness: 0.56838 stdev: 0.32629
Best fitness: 0.89103 - size: (12, 13) - species 5 - id 2961
Average adjusted fitness: 0.555
Mean genetic distance 2.973, standard deviation 0.538
Population of 196 members in 15 species
Total extinctions: 0
Generation time: 50.207 sec (44.586 average)

 ****** Running generation 25 ****** 

Population's average fitness: 0.53292 stdev: 0.33454
Best fitness: 0.89103 - size: (12, 13) - species 5 - id 2961
Average adjusted fitness: 0.512
Mean genetic distance 2.984, standard deviation 0.523
Population of 199 members in 16 species
Total extinctions: 0
Generation time: 45.744 sec (44.868 average)

 ****** Running generation 26 ****** 

Population's average fitness: 0.55269 stdev: 0.33164
Best fitness: 0.89103 - size: (12, 13) - species 5 - id 2961
Average adjusted fitness: 0.549
Mean genetic distance 2.958, standard deviation 0.522
Population of 199 members in 16 species
Total extinctions: 0
Generation time: 48.695 sec (45.747 average)

 

Population's average fitness: 0.62211 stdev: 0.29858
Best fitness: 0.91517 - size: (15, 18) - species 5 - id 7052
Average adjusted fitness: 0.542
Mean genetic distance 3.154, standard deviation 0.550
Population of 196 members in 16 species
Total extinctions: 0
Generation time: 61.563 sec (54.434 average)

 ****** Running generation 49 ****** 

Population's average fitness: 0.60789 stdev: 0.31407
Best fitness: 0.91517 - size: (15, 18) - species 5 - id 7052
Average adjusted fitness: 0.560
Mean genetic distance 3.190, standard deviation 0.545
Population of 197 members in 16 species
Total extinctions: 0
Generation time: 58.537 sec (55.510 average)

 ****** Running generation 50 ****** 

Population's average fitness: 0.57648 stdev: 0.32619
Best fitness: 0.91517 - size: (15, 18) - species 5 - id 7052
Average adjusted fitness: 0.536
Mean genetic distance 3.177, standard deviation 0.556
Population of 202 members in 16 species
Total extinctions: 0
Generation time: 54.818 sec (56.137 average)

 

Population's average fitness: 0.63136 stdev: 0.30490
Best fitness: 0.91770 - size: (15, 19) - species 5 - id 9100
Average adjusted fitness: 0.623
Mean genetic distance 3.125, standard deviation 0.575
Population of 200 members in 15 species
Total extinctions: 0
Generation time: 66.040 sec (58.212 average)

 ****** Running generation 73 ****** 

Population's average fitness: 0.63255 stdev: 0.30365
Best fitness: 0.91816 - size: (16, 23) - species 5 - id 12103
Average adjusted fitness: 0.621
Mean genetic distance 3.126, standard deviation 0.574
Population of 201 members in 15 species
Total extinctions: 0
Generation time: 59.627 sec (58.736 average)

 ****** Running generation 74 ****** 

Population's average fitness: 0.63135 stdev: 0.29669
Best fitness: 0.91816 - size: (16, 23) - species 5 - id 12103
Average adjusted fitness: 0.611
Mean genetic distance 3.141, standard deviation 0.568
Population of 195 members in 15 species
Total extinctions: 0
Generation time: 60.252 sec (59.005 average)


Population's average fitness: 0.61274 stdev: 0.31269
Best fitness: 0.91816 - size: (21, 28) - species 46 - id 14493
Average adjusted fitness: 0.620
Mean genetic distance 3.063, standard deviation 0.533
Population of 195 members in 20 species
Total extinctions: 0
Generation time: 74.127 sec (68.707 average)

 ****** Running generation 97 ****** 

Population's average fitness: 0.64055 stdev: 0.29538
Best fitness: 0.91816 - size: (21, 28) - species 46 - id 14493
Average adjusted fitness: 0.643
Mean genetic distance 3.063, standard deviation 0.531
Population of 194 members in 19 species
Total extinctions: 0
Generation time: 73.610 sec (69.554 average)

 ****** Running generation 98 ****** 

Population's average fitness: 0.65614 stdev: 0.28255
Best fitness: 0.91816 - size: (21, 28) - species 46 - id 14493
Average adjusted fitness: 0.667
Mean genetic distance 3.083, standard deviation 0.542
Population of 194 members in 18 species
Total extinctions: 0
Generation time: 68.535 sec (69.524 avera

...

Mean genetic distance 2.698, standard deviation 0.427
Population of 203 members in 11 species
Total extinctions: 0
Generation time: 65.560 sec (57.814 average)

 ****** Running generation 1696 ****** 

Population's average fitness: 0.57346 stdev: 0.32911
Best fitness: 0.92782 - size: (38, 58) - species 131 - id 125421
Average adjusted fitness: 0.568
Mean genetic distance 2.713, standard deviation 0.434
Population of 202 members in 11 species
Total extinctions: 0
Generation time: 55.164 sec (58.050 average)

 ****** Running generation 1697 ****** 

Population's average fitness: 0.60833 stdev: 0.32204
Best fitness: 0.92782 - size: (38, 58) - species 131 - id 125421
Average adjusted fitness: 0.603
Mean genetic distance 2.755, standard deviation 0.446
Population of 201 members in 11 species
Total extinctions: 0
Generation time: 55.196 sec (58.152 average)

 ****** Running generation 1698 ****** 

Population's average fitness: 0.58795 stdev: 0.31849
Best fitness: 0.92782 - size: (38, 58) - 

Population's average fitness: 0.63785 stdev: 0.29319
Best fitness: 0.92782 - size: (38, 58) - species 131 - id 125421
Average adjusted fitness: 0.635
Mean genetic distance 2.780, standard deviation 0.434
Population of 198 members in 13 species
Total extinctions: 0
Generation time: 30.890 sec (32.895 average)

 ****** Running generation 1720 ****** 

Population's average fitness: 0.61561 stdev: 0.29881
Best fitness: 0.92782 - size: (38, 58) - species 131 - id 125421
Average adjusted fitness: 0.618
Mean genetic distance 2.787, standard deviation 0.399
Population of 197 members in 13 species
Total extinctions: 0
Generation time: 34.614 sec (33.045 average)

 ****** Running generation 1721 ****** 

Population's average fitness: 0.65713 stdev: 0.29406
Best fitness: 0.92782 - size: (38, 58) - species 131 - id 125421
Average adjusted fitness: 0.659
Mean genetic distance 2.796, standard deviation 0.407
Population of 198 members in 13 species
Total extinctions: 0
Generation time: 34.153 sec (32

Mean genetic distance 2.845, standard deviation 0.445
Population of 199 members in 12 species
Total extinctions: 0
Generation time: 31.613 sec (32.117 average)

 ****** Running generation 1743 ****** 

Population's average fitness: 0.62370 stdev: 0.30691
Best fitness: 0.92782 - size: (38, 58) - species 131 - id 125421
Average adjusted fitness: 0.622
Mean genetic distance 2.828, standard deviation 0.444
Population of 202 members in 12 species
Total extinctions: 0
Generation time: 31.738 sec (32.109 average)

 ****** Running generation 1744 ****** 

Population's average fitness: 0.61835 stdev: 0.30839
Best fitness: 0.92782 - size: (38, 58) - species 131 - id 125421
Average adjusted fitness: 0.616
Mean genetic distance 2.839, standard deviation 0.440
Population of 204 members in 12 species
Total extinctions: 0
Generation time: 32.443 sec (32.243 average)

 ****** Running generation 1745 ****** 

Population's average fitness: 0.59115 stdev: 0.31789
Best fitness: 0.92782 - size: (38, 58) - 

Population's average fitness: 0.58322 stdev: 0.33202
Best fitness: 0.92782 - size: (38, 58) - species 131 - id 125421
Average adjusted fitness: 0.595
Mean genetic distance 2.863, standard deviation 0.404
Population of 201 members in 16 species
Total extinctions: 0
Generation time: 37.996 sec (187.728 average)

 ****** Running generation 1767 ****** 

Population's average fitness: 0.58455 stdev: 0.32702
Best fitness: 0.92782 - size: (38, 58) - species 131 - id 125421
Average adjusted fitness: 0.576
Mean genetic distance 2.857, standard deviation 0.410
Population of 200 members in 16 species
Total extinctions: 0
Generation time: 37.857 sec (188.342 average)

 ****** Running generation 1768 ****** 

Population's average fitness: 0.65625 stdev: 0.28854
Best fitness: 0.92782 - size: (38, 58) - species 131 - id 125421
Average adjusted fitness: 0.639
Mean genetic distance 2.861, standard deviation 0.420
Population of 195 members in 15 species
Total extinctions: 0
Generation time: 38.907 sec (

Mean genetic distance 2.858, standard deviation 0.467
Population of 200 members in 12 species
Total extinctions: 0
Generation time: 78.126 sec (78.036 average)

 ****** Running generation 1790 ****** 

Population's average fitness: 0.59729 stdev: 0.33019
Best fitness: 0.92782 - size: (38, 58) - species 131 - id 125421
Average adjusted fitness: 0.611
Mean genetic distance 2.879, standard deviation 0.451
Population of 199 members in 12 species
Total extinctions: 0
Generation time: 85.212 sec (80.206 average)

 ****** Running generation 1791 ****** 

Population's average fitness: 0.64954 stdev: 0.29548
Best fitness: 0.92782 - size: (38, 58) - species 131 - id 125421
Average adjusted fitness: 0.652
Mean genetic distance 2.875, standard deviation 0.466
Population of 199 members in 12 species
Total extinctions: 0
Generation time: 77.938 sec (80.195 average)

 ****** Running generation 1792 ****** 

Population's average fitness: 0.63852 stdev: 0.29833
Best fitness: 0.92782 - size: (38, 58) - 

In [13]:
local_dir = os.getcwd()
config_path = os.path.join(local_dir, "config-feedforward-satellite6")

config = neat.Config(
    neat.DefaultGenome,
    neat.DefaultReproduction,
    neat.DefaultSpeciesSet,
    neat.DefaultStagnation,
    config_path,
)

In [14]:
test_df = pd.read_csv('sat.tst',header=None,sep = '\s')

testX = test_df.iloc[:,:-1].to_numpy()
testy = test_df.iloc[:,-1].to_numpy()

# 7-> 6 and 1-6 -> 0:5
testy[testy==7] = 6
for i in range(6):
    testy[testy == (i + 1)] = i

In [15]:
winner_net

In [19]:
test_x_inputs =[tuple(c) for c in testX]
test_x_outputs = [tuple([c]) for c in testy]

In [32]:
testsamplesize = 2000
outputs = []
for xi in test_x_inputs:
    output = winner_net.activate(xi)
    outputs.append(output)

px_outputs = softmax(np.array(outputs).reshape(testsamplesize, 6), axis=1)
# the index of maximum in each line
pred_outputs = np.argmax(px_outputs, axis = 1)
real_outputs = np.array(test_x_outputs).reshape(testsamplesize,)

acc = np.sum(pred_outputs == real_outputs)/testsamplesize
print("Accuracy: {}".format(acc))

Accuracy: 0.5645
